# Tutorial 07: Open shells

In this tutorial you will learn more about how psi4 handles open-shell species

In [ ]:
import psi4

When doing Hartree–Fock or DFT computations on closed-shell species (even number of electrons, all paired) psi4 automatically selectes a restricted formalism. This means that the spatial part of alpha and beta orbitals is the same
$$
\phi_{i\alpha}(\mathbf{r}) = \phi_{i\beta}(\mathbf{r})
$$
This implies that the alpha and beta orbital energies are identical as a simple computation confirms

In [2]:
psi4.geometry(f"""
H
F 1 1.0
symmetry c1
""")

e, wfn = psi4.energy('HF/def2-SVP',return_wfn=True)

In [3]:
print(wfn.epsilon_a().to_array()) # alpha orbital energies
print(wfn.epsilon_b().to_array()) # beta orbital energies

[-26.26971571  -1.53019894  -0.71818034  -0.62651052  -0.62651052
   0.15830265   0.73941872   1.41561931   1.41561931   1.43621331
   1.65591555   1.71388604   1.71388604   2.41789009   3.48489998
   3.48489998   3.7312377    3.7312377    4.55627241]
[-26.26971571  -1.53019894  -0.71818034  -0.62651052  -0.62651052
   0.15830265   0.73941872   1.41561931   1.41561931   1.43621331
   1.65591555   1.71388604   1.71388604   2.41789009   3.48489998
   3.48489998   3.7312377    3.7312377    4.55627241]


## Simple open shells

Suppose that we are now interested in studying the H$_2^+$ molecule. This system has one unpaired electron, which implies that the spin of the H$_2^+$ molecule is nonzero.
A single electron has a total spin quantum number $s = 1/2$ corresponding to the operator $\hat{S}^2$, which means
$$
\hat{S}^2 \sigma = s(s+1) \sigma, \text{ where } \sigma = \alpha \text{ or } \beta
$$
The z component of spin (spin projected on the z axis) is the quantum number $m_s$ and it is equal to
$$
m_s = +\frac{1}{2} \text{ or } -\frac{1}{2} \text{ (alpha or beta)}.
$$
This quantum number corresponds to the operator $\hat{S}_z$, which means
$$
\hat{S}_z \sigma = m_s \sigma, \text{ where } \sigma = \alpha \text{ or } \beta
$$

Since there are two possiblities for the spin of the electron, we have a **multiplicity equal to 2**. Another way to compute the multiplicity is via the equation
$$
\text{multiplicity } = 2s + 1
$$
Let's now try to compute the Hartree–Fock energy of H$_2^+$ with psi4

## Restricted vs. unrestricted methods

There are two ways to perform computations on open shell systems like H$_2^+$ and we need to specify one.

1. using a restricted formalism that enforces the condition
$$
\phi_{i\alpha}(\mathbf{r}) = \phi_{i\beta}(\mathbf{r})
$$
for all doubly occupied orbitals.
1. using an unrestricted formalism in which we let alpha and beta orbitals have different spatial part
$$
\phi_{i\alpha}(\mathbf{r}) \neq \phi_{i\beta}(\mathbf{r})
$$
for both doubly and singly occupied orbitals.


Note that **for a single electron, these two approaches give the same energy**.

To perform a restricted Hartree–Fock computation you neet to set the option `REFERENCE` to `ROHF`.

In [8]:
psi4.geometry("""
1 2
H
H 1 1.0
symmetry c1
""")

psi4.set_options({'REFERENCE' : 'ROHF'})


e, wfn = psi4.energy('HF/def2-SVP',return_wfn=True)

Now let us check the orbital energies. In the case of ROHF these might be defined differently by different programs because there is not a unique way to define them. Psi4 yields equal energies for the alpha and beta orbitals

In [9]:
print(wfn.epsilon_a().to_array()) # alpha orbital energies
print(wfn.epsilon_b().to_array()) # beta orbital energies

[-0.78975315 -0.21148354  0.14907738  0.3768626   0.9197362   0.9197362
  1.26681617  1.52228616  1.52228616  2.41856805]
[-0.78975315 -0.21148354  0.14907738  0.3768626   0.9197362   0.9197362
  1.26681617  1.52228616  1.52228616  2.41856805]


An unrestricted HF computation can be done by setting the option `REFERENCE` to `UHF`.

In [10]:
psi4.geometry("""
1 2
H
H 1 1.0
symmetry c1
""")

psi4.set_options({'REFERENCE' : 'UHF'})

e, wfn = psi4.energy('HF/def2-SVP',return_wfn=True)
print(wfn.epsilon_a().to_array()) # alpha orbital energies
print(wfn.epsilon_b().to_array()) # beta orbital energies

[-1.12873752 -0.26617959  0.11742833  0.32905274  0.8639872   0.8639872
  1.22168315  1.49209801  1.49209801  2.38879522]
[-0.48162337 -0.1639748   0.20884091  0.43134234  0.9754852   0.9754852
  1.31468928  1.5524743   1.5524743   2.44885832]


Note that the energies of the alpha and beta electrons are different. In the unrestricted case the alpha and beta orbitals are different and we can see this if we plot them.

In [11]:
!mkdir mos

psi4.cubeprop(wfn)

import fortecubeview
fortecubeview.plot()

CubeViewer: listing cube files from the directory .
Reading 20 cube files


HTML(value='\n        <style>\n           .jupyter-widgets-output-area .output_scroll {\n                heigh…

interactive(children=(Select(description='Cube files:', options=('MO    1a (1-A)', 'MO    1b (1-A)', 'MO    2a…

Note that **in UHF the orbitals may be different but this is not always the case**. For example, if you apply UHF to a closed shell system then often you recover the RHF solution. For example

In [12]:
psi4.geometry("""
0 1
H
F 1 1.0
symmetry c1
""")

psi4.set_options({'REFERENCE' : 'UHF'})
e_uhf = psi4.energy('HF/def2-SVP')

psi4.set_options({'REFERENCE' : 'RHF'})
e_rhf = psi4.energy('HF/def2-SVP')

print(f'The difference between the RHF and UHF energy is {e_rhf - e_uhf}')

The difference between the RHF and UHF energy is 0.0


## General case

When we have more than one electron, we use capital letters to refer to the sum of the angular momentum of all electrons. The quantum number $M_S$ is the sum of the z component of spin for all the electrons, which is given by the difference between the number of alpha and beta electrons divided by 2
$$
M_S = \frac{1}{2} (N_\alpha - N_\beta)
$$

An important result in the theory of angular momentum is that a system with total spin equal to $S$ has multiplicity = $2S + 1$, which means that there are $2S + 1$ states with the same total spin. These states have z-projection of spin that ranges from $-S$ to $S$ in increments of 1
$$
-S, -S + 1, -S + 2, \ldots, S - 2, S-1, S
$$

Now consider the case of HF$^+$. This molecule has 5 alpha and 4 beta electrons (or the other way around).
For HF$^+$ has $M_S = 1/2$ which comes from the 5 alpha and 4 beta electrons ((5 - 4)/2).

For HF$^+$ this means that the state with one unpaired alpha electron corresponds to $S = 1/2$. If it was otherwise things would not work out. For example, if $S = 1$ then we should find state with 
$$
M_S = -1, 0, +1
$$
but we know that with one unpaired electron we need to have a state with $M_S = 1/2$.
If we postulated that $S = 3/2$ then we should be able to find states with
$$
M_S = -3/2, -1/2, 1/2 , 3/2
$$
However, with only one unpaired electron we cannot have a state with $M_S = 3/2$.

To summarize, the HF$^+$ molecule has multiplicity equal to $2S + 1 = 2 1/2 + 1 = 2$.

## Spin contamination of unrestricted wave functions

ROHF and UHF generally yield different energies

In [28]:
psi4.geometry("""
1 2
H
F 1 1.0
symmetry c1
""")

psi4.set_options({'REFERENCE' : 'ROHF'})
e_rohf = psi4.energy('HF/def2-SVP')

psi4.set_output_file('uhf.txt',False)

psi4.set_options({'REFERENCE' : 'UHF'})
e_uhf = psi4.energy('HF/def2-SVP')

print(f'E(ROHF) = {e_rohf} Eh')
print(f'E(UHF)  = {e_uhf}  Eh')
print(f'The difference between the ROHF and UHF energy is {e_rohf - e_uhf}')

E(ROHF) = -99.40734247254734 Eh
E(UHF)  = -99.4115915867321  Eh
The difference between the ROHF and UHF energy is 0.004249114184759151


Why is there an energy difference between the ROHF and UHF? And why is UHF the lower solution? Let's check the output

In [33]:
with open('uhf.txt') as f:
    print(''.join(f.readlines()[130:]))

   @DF-UHF iter   6:   -99.41158993908849   -1.29086e-05   5.22563e-05 DIIS
   @DF-UHF iter   7:   -99.41159154518073   -1.60609e-06   9.26010e-06 DIIS
   @DF-UHF iter   8:   -99.41159158594449   -4.07638e-08   1.60858e-06 DIIS
   @DF-UHF iter   9:   -99.41159158673210   -7.87608e-10   2.35384e-07 DIIS
  Energy and wave function converged.


  ==> Post-Iterations <==

   @Spin Contamination Metric:   4.248071198E-03
   @S^2 Expected:                7.500000000E-01
   @S^2 Observed:                7.542480712E-01
   @S   Expected:                5.000000000E-01
   @S   Observed:                5.000000000E-01

    Orbital Energies [Eh]
    ---------------------

    Alpha Occupied:                                                       

       1A    -26.964701     2A     -2.206046     3A     -1.369701  
       4A     -1.337649     5A     -1.255858  

    Alpha Virtual:                                                        

       6A     -0.208760     7A      0.310492     8A      0.827

In [3]:
import psi4 
psi4.geometry(f"""
0 3
C
H 1 1.0
H 1 1.0 2 104.5
symmetry c1
""")

psi4.set_options({'REFERENCE' : 'UKS'})
e, wfn = psi4.optimize('B3LYP/def2-SVP',return_wfn=True)

Optimizer: Optimization complete!


In [10]:
!mkdir ch2
psi4.set_options({'CUBEPROP_FILEPATH' : './ch2','CUBEPROP_TASKS' : ['DENSITY']})
psi4.cubeprop(wfn)

mkdir: ch2: File exists


In [19]:
!ls ch2
# !rm ch2/Ps*

import fortecubeview
fortecubeview.plot(path='./ch2',sumlevel=0.9)

Da.cube  Db.cube  Ds.cube  Dt.cube  geom.xyz
CubeViewer: listing cube files from the directory ./ch2
Reading 4 cube files


HTML(value='\n        <style>\n           .jupyter-widgets-output-area .output_scroll {\n                heigh…

interactive(children=(Select(description='Cube files:', options=('Density (alpha)', 'Density (beta)', 'Density…